In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install surprise

     |████████████████████████████████| 11.8MB 230kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617614 sha256=f51c74860c456bd6c95cb2a43697d2259e0798fb1c336ac92c07b222919a69f0
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [10]:
rating = pd.read_csv('ratings.csv', sep=',')
movie = pd.read_csv('movies.csv', sep=',')

metricnya pake userid,movieid/productid, jumlah rating = resultnya top-N movies


In [11]:
rating.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [12]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
df = rating[["userId","movieId","rating"]] #milih data apa aja yg mau diambil (custom dataset)
df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [17]:
min_rating = df["rating"].min()
max_rating = df["rating"].max()

print("Range rating: {0} to {1}".format(min_rating, max_rating))

Range rating: 0.5 to 5.0


BOOST PERFORMA MODEL ML dengan Hyperparameter Tuning

In [23]:
from surprise import Reader
from surprise import Dataset
from surprise import SVD
from surprise.model_selection import GridSearchCV

# load data ke format bawaan surprise
reader = Reader(rating_scale=(min_rating, max_rating))
data = Dataset.load_from_df(df, reader = reader)

param_grid = {
    'n_epochs': [5, 10], 
    'n_factors': [20, 50, 100]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)
gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.8777457442176335
{'n_epochs': 10, 'n_factors': 20}


In [26]:
#nyimpen value factor ke dalam variable
best_factor = gs.best_params['rmse']['n_factors']
best_factor

20

In [28]:
#nyimpen value epoch ke dalam variable
best_epoch = gs.best_params['rmse']['n_epochs']
best_epoch

10

LANJUT KE TOP-N Recomend

In [29]:
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy

# load data ke format bawaan surprise
reader = Reader(rating_scale=(min_rating, max_rating))
data = Dataset.load_from_df(df, reader = reader)

# Bagi dataset menjadi 20:80
trainset, testset = train_test_split(data, test_size=.20)

#Gunakan algo SVD
algo = SVD(n_factors=best_factor, n_epochs=best_epoch)

#mulai latihan 
algo.fit(trainset)

In [43]:
# Pilih User ID yang ingin direkomendasikan
userID = 2

In [32]:
df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [33]:
# Dapet List semua movie ID dari dataset
movie_ids = df["movieId"].unique()
movie_ids

array([     1,      3,      6, ..., 160836, 163937, 163981])

In [44]:
# Dapetin List movie yang sudah pernah dilihat user
movie_ids_user = df.loc[df["userId"] == userID, "movieId"]
movie_ids_user

232       318
233       333
234      1704
235      3578
236      6874
237      8798
238     46970
239     48516
240     58559
241     60756
242     68157
243     71535
244     74458
245     77455
246     79132
247     80489
248     80906
249     86345
250     89774
251     91529
252     91658
253     99114
254    106782
255    109487
256    112552
257    114060
258    115713
259    122882
260    131724
Name: movieId, dtype: int64

In [45]:
# Cari selisih antara movie yang ada dengan movie yang sudah dilihat user
movie_ids_to_prediction = np.setdiff1d(movie_ids, movie_ids_user)

In [47]:
#Buat test set dengan metric prediksi user id dan movie id
test_set = [[userID, movie_id, 4] for movie_id in movie_ids_to_prediction]

In [49]:
#mulai prediksi
predictions = algo.test(test_set)

#cari rata rata prediksi
predictions[0]

Prediction(uid=2, iid=1, r_ui=4, est=4.059047767444006, details={'was_impossible': False})

In [50]:
# prediction estimation akan menjadi dasar dari mencari rekomendasi
pred_ratings = np.array([pred.est for pred in predictions])
pred_ratings

array([4.05904777, 3.76251503, 3.44424127, ..., 3.61803987, 3.6328019 ,
       3.7638184 ])

In [55]:
n_items = 10

In [59]:
print("Top {0} item recommendations for user {1}:".format(n_items, userID))
index_max = (-pred_ratings).argsort()[:n_items]
for i in index_max:
    movie_id = movie_ids_to_prediction[i]
    print(movie[movie["movieId"]==movie_id]["title"].values[0], pred_ratings[i])

Top 10 item recommendations for user 2:
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) 4.505673747839341
Godfather: Part II, The (1974) 4.447499012836327
Apocalypse Now (1979) 4.443461202490077
Monty Python and the Holy Grail (1975) 4.436203921947562
Fight Club (1999) 4.414217346818807
Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001) 4.40614803008992
Godfather, The (1972) 4.400774068196461
Matrix, The (1999) 4.39449148175108
Usual Suspects, The (1995) 4.385192328755067
Amadeus (1984) 4.384928441121724
